# Generate Linear Acceleration and Angular Velocity Data

In order to test the function for calculating the trajectory it seems to be necessary to generate own data.

In [1]:
import skinematics as skin
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import functionsMasterProjectMeinhart as fmpm

In [2]:
%matplotlib auto

Using matplotlib backend: TkAgg


In [3]:
g_mps2 = 9.80665 # [m/s^2]
sampling_rate = 100 # [Hz]
delta_t = 1/ sampling_rate

num_data_points = 1000

# reserve space for 10 s data (due to sampling rate of 100 Hz)
acc_lin = np.zeros((num_data_points,3))
time = np.arange(np.shape(acc_lin)[0])/sampling_rate

q0 = np.array([0,0,0])
q_vec = np.zeros((num_data_points + 1,3)) # +1 because of initial orientation

# define angular velocity in rad/s 
omega = np.tile([0,90,0], (num_data_points,1))*np.pi/180

## Let's generate the acceleration profile for a rotating sensor, only influenced by gravity

In [4]:
acc_lin_start = [0,0,1] # starting position

In [5]:
#quat = skin.quat.calc_quat(omega, q0, rate=sampling_rate, CStype='bf')

In [6]:
for ii in range(np.shape(omega)[0]):
    
    omega_abs = np.linalg.norm(omega[ii])
        
    if omega_abs < 0.00001 and omega_abs > -0.00001:
        q_vec[ii+1] = [0,0,0]

    else:
        delta_q_vec = omega[ii]/omega_abs * np.sin(omega_abs*delta_t/2)
        q_vec[ii+1] = skin.quat.q_mult(q_vec[ii], delta_q_vec)

In [7]:
rotmat_set = skin.quat.convert(q_vec[1:], to='rotmat')

In [8]:
acc = np.array([np.dot(np.matrix.transpose(rotmat_set[ii].reshape((3,3))), acc_lin_start) for ii in range(len(rotmat_set ))])

In [9]:
plt.figure()
fmpm.plot_signal(acc,time)

In [10]:
plt.figure()
fmpm.plot_signal(q_vec[1:],time,
            Title='Vector Part of Quaternion',
            xLabel=r'$time \enspace [s]$',
            yLabel=r'$q$')

In [11]:
acc

array([[-1.57073173e-02,  0.00000000e+00,  9.99876632e-01],
       [-3.14107591e-02,  0.00000000e+00,  9.99506560e-01],
       [-4.71064507e-02,  0.00000000e+00,  9.98889875e-01],
       ...,
       [-3.14107889e-02,  0.00000000e+00, -9.99506559e-01],
       [-1.57073471e-02,  0.00000000e+00, -9.99876632e-01],
       [-2.98023224e-08,  0.00000000e+00, -1.00000000e+00]])

In [12]:
traj_data = fmpm.calc_trajectory_orientation(acc,
                                            omega*180/np.pi,
                                            acc_init=acc_lin_start,
                                            sampling_rate=sampling_rate)

In [13]:
plt.figure()
fmpm.plot_signal(traj_data['q_vec'][1:],time,
            Title='Vector Part of Quaternion',
            xLabel=r'$time \enspace [s]$',
            yLabel=r'$q$')

In [14]:
[pos_x, pos_y, pos_z] = [traj_data['pos'][:,ii] for ii in range(3)]

%matplotlib auto
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot(pos_x, pos_y, pos_z, label='trajectory')
ax.set_xlabel(r'$x [m]$')
ax.set_ylabel(r'$y [m]$')
ax.set_zlabel(r'$z [m]$')
ax.legend()
plt.show()

Using matplotlib backend: TkAgg


## Let's add acceleration in the beginning for 1 second

In [31]:
acc_total_begin = acc_lin_start + np.array([1,1,0])/np.sqrt(2)/g_mps2 # + 1m/s^2
acc_first_sec = np.tile(acc_total_begin, (sampling_rate,1)) # data for 1 sec

In [32]:
# no angular velocity in the first second
omega_first_sec = np.tile([0,0,0], (sampling_rate,1))

In [33]:
acc_new = np.vstack((acc_first_sec, acc))
omega_new = np.vstack((omega_first_sec, omega))
time_new = np.arange(np.shape(acc_new)[0])/sampling_rate

In [34]:
traj_data = fmpm.calc_trajectory_orientation(acc_new,
                                            omega_new*180/np.pi,
                                            acc_init=acc_lin_start,
                                            sampling_rate=sampling_rate)

In [35]:
plt.figure()
fmpm.plot_signal(traj_data['q_vec'][:-1],
                 time_new,
                 Title='Vector Part of Quaternion',
                 xLabel=r'$time \enspace [s]$',
                 yLabel=r'$q$')

In [36]:
[pos_x, pos_y, pos_z] = [traj_data['pos'][:,ii] for ii in range(3)]

%matplotlib auto
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot(pos_x, pos_y, pos_z, label='trajectory')
ax.set_xlabel(r'$x [m]$')
ax.set_ylabel(r'$y [m]$')
ax.set_zlabel(r'$z [m]$')
ax.legend()
plt.show()

Using matplotlib backend: TkAgg


In [16]:
def calc_trajectory_orientation(acc_lin_g, vel_ang_degps, acc_init, sampling_rate=256):
    '''
    This function calculates the trajectory by means of a set of linear acceleration and angular velocity data.
    The sampling rate has to be given in Hz.
    Moreover, the initial acceleration is taken as gravity.
    This allows to calculate the position in an upright frame (z-axis aligned with gravity).
    
    Inputs
    ------
    acc_lin_g: linear acceleration [g]
    
    vel_ang_degps: angular velocity [deg/s]
    
    acc_init: initial linear acceleration, shall be only caused by gravity
    
    sampling_rate: sampling rate of the measured signals in Hz
    
    
    Returns
    -------
    Dictionary with upright position, velocity and orientation (vector-part of quaternion)
    '''
    
    g_mps2 = 9.80665 # [m/s^2]
    
    acc_lin = np.array(acc_lin_g) * g_mps2 # linear acceleration in m/s^2
    
    vel_ang = np.array(vel_ang_degps) * np.pi/180 # angular velocity in rad/s
    
    delta_t = 1/sampling_rate
    
    
    # initial acceleration shall be only caused by gravity
    acc_init_abs = np.linalg.norm(acc_init)
    acc_init_norm = np.array(acc_init) / acc_init_abs # normalize
    
    beta = np.arcsin(acc_init_norm[0]) # rotation angle (y)
    alpha = np.arccos(acc_init_norm[2]/np.cos(beta)) # rotation angle (x)
    
    R_upright_init = np.matrix.transpose(np.array([[np.cos(beta), 0, np.sin(beta)],
       [-np.sin(alpha)*np.sin(beta), np.cos(alpha), -np.sin(alpha)*np.cos(beta)],
       [np.sin(beta)*np.cos(alpha), np.sin(alpha), np.cos(alpha)*np.cos(beta)]]))


    acc_upright_gravity = np.array([0, 0, g_mps2]) # gravity in upright position [m/s^2]
    
    # check number of data points
    if acc_lin.ndim is 1 or vel_ang.ndim is 1:
        num_steps = 1
    elif np.shape(acc_lin)[0] <= np.shape(vel_ang)[0]:
        num_steps = np.shape(acc_lin)[0]
    else:
        num_steps = np.shape(vel_ang)[0]
    
    # reserve memory for position and orientation vectors
    #    --> one step more, because of initial condition
    pos_upright = np.zeros((num_steps+1, 3))
    vel_upright = np.zeros((num_steps+1, 3))
    q_vec = np.zeros((num_steps+1, 3)) # vector part of quaternion
         
    
    
    for ii in range(num_steps):
        
        omega_abs = np.linalg.norm(vel_ang[ii])
        
        # avoid division by zero (or value close to zero)
        if omega_abs < 0.00001 and omega_abs > -0.00001:
            q_vec[ii+1] = q_vec[ii]
        
        else:
            delta_q_vec = vel_ang[ii]/omega_abs * np.sin(omega_abs*delta_t/2)
            q_vec[ii+1] = skin.quat.q_mult(q_vec[ii], delta_q_vec)
        
        R_space_obj_new = skin.quat.convert(q_vec[ii+1] , to='rotmat')
        
        acc_upright_movement = np.dot(R_upright_init, np.dot(R_space_obj_new, acc_lin[ii])) - acc_upright_gravity
    
        vel_upright[ii+1] = vel_upright[ii] + acc_upright_movement*delta_t

        pos_upright[ii+1] = pos_upright[ii] +  vel_upright[ii]*delta_t + 0.5*acc_upright_movement*delta_t**2
        
        
    data_dict = {}
    data_dict['pos'] = pos_upright
    data_dict['vel'] = vel_upright
    data_dict['q_vec'] = q_vec
    
    return data_dict